In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import baskerville_dna as dna
import pysam
import h5py
from borzoi_pytorch.data import process_sequence, bin_methylation

In [2]:
genome = pysam.FastaFile('/home/tobyc/data/borzoi-pytorch/data/ref_genomes/GRCh37/GCF_000001405.13/GCF_000001405.13_GRCh37_genomic.fna')

## Processing data from paper (GSE41169) into a usable format

In [24]:
data_csv = pd.read_csv('../data/me_chip/GSE41169_series_matrix.txt', delimiter='\t', skiprows=72, skipfooter=1, engine='python')

/tmp/ipykernel_51866/1967789236.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data_csv = pd.read_csv('../data/me_chip/GSE41169_series_matrix.txt', delimiter='\t', skiprows=72, skipfooter=1)


In [26]:
data_csv.set_index('ID_REF', inplace=True)

In [36]:
data_csv.mean(axis=1).reset_index().rename(columns={0:'mean_beta'}).to_csv('../data/me_chip/mean_beta_values.csv', index=False)

In [38]:
# Mapping data to reference genome:

# Load your beta-values (probes x samples)
beta_df = pd.read_csv("../data/me_chip/mean_beta_values.csv")

# Load the manifest
manifest = pd.read_csv("../data/me_chip/manifest/GPL13534_HumanMethylation450_15017482_v.1.1.csv", skiprows=7)

# Keep only relevant columns from manifest
manifest_sub = manifest[['IlmnID', 'CHR', 'MAPINFO', 'Forward_Sequence']]

# Merge beta-values with manifest annotation
# Assume beta_df first column is 'ProbeID'
beta_df.rename(columns={beta_df.columns[0]: 'IlmnID'}, inplace=True)

annotated = pd.merge(manifest_sub, beta_df, on='IlmnID', how='inner')

# Now 'annotated' contains: probe ID, chromosome, position, sequence + beta-values
print(annotated.head())


/tmp/ipykernel_51866/3642129388.py:7: DtypeWarning: Columns (2,4,11,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  manifest = pd.read_csv("../data/me_chip/manifest/GPL13534_HumanMethylation450_15017482_v.1.1.csv", skiprows=7)


       IlmnID CHR     MAPINFO  \
0  cg00035864   Y   8553009.0   
1  cg00050873   Y   9363356.0   
2  cg00061679   Y  25314171.0   
3  cg00063477   Y  22741795.0   
4  cg00121626   Y  21664296.0   

                                    Forward_Sequence  mean_beta  
0  AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCT...   0.245431  
1  TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTGGTGGTTTTTG...   0.784335  
2  TCAACAAATGAGAGACATTGAAGAACTAATTCACTACTATTTGGTT...   0.580761  
3  CTCCTGTACTTGTTCATTAAATAATGATTCCTTGGATATACCAAGT...   0.795170  
4  AGGTGAATGAAGAGACTAATGGGAGTGGCTTGCAAGCCAGGTACTG...   0.398251  


In [43]:
annotated.isna().sum()

IlmnID               0
CHR                 65
MAPINFO             65
Forward_Sequence    65
mean_beta            0
dtype: int64

In [46]:
annotated.dropna(inplace=True)
annotated['MAPINFO'] = annotated['MAPINFO'].astype(int)

In [89]:
annotated['CHR'] = annotated['CHR'].astype(str)

In [90]:
annotated

,IlmnID,CHR,MAPINFO,Forward_Sequence,mean_beta
0,cg00035864,Y,8553009,AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCT...,0.245431
1,cg00050873,Y,9363356,TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTGGTGGTTTTTG...,0.784335
2,cg00061679,Y,25314171,TCAACAAATGAGAGACATTGAAGAACTAATTCACTACTATTTGGTT...,0.580761
3,cg00063477,Y,22741795,CTCCTGTACTTGTTCATTAAATAATGATTCCTTGGATATACCAAGT...,0.795170
4,cg00121626,Y,21664296,AGGTGAATGAAGAGACTAATGGGAGTGGCTTGCAAGCCAGGTACTG...,0.398251
...,...,...,...,...,...
485507,ch.22.909671F,22.0,46114168,TTTTCCTTTTAGCTGCTGATAGATTAATAGTATGTGAACCTTTTAA...,0.056644
485508,ch.22.46830341F,22.0,48451677,TGTGCATACATGCGCATGTGAACAGTCCATGGAGCTTAATCCCCTG...,0.026932
485509,ch.22.1008279F,22.0,48731367,CTGGCAGGGCACACACCTCAGCTGGGCCCTGTGGCAGGTGAACCCC...,0.020596
485510,ch.22.47579720R,22.0,49193714,ATGTACCCATACGGGAAAGGCCGCGTGAAGATGGAGACAGAGATGG...,0.064398


In [91]:
annotated['CHR'] = annotated['CHR'].str.replace('.0','')
annotated

,IlmnID,CHR,MAPINFO,Forward_Sequence,mean_beta
0,cg00035864,Y,8553009,AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCT...,0.245431
1,cg00050873,Y,9363356,TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTGGTGGTTTTTG...,0.784335
2,cg00061679,Y,25314171,TCAACAAATGAGAGACATTGAAGAACTAATTCACTACTATTTGGTT...,0.580761
3,cg00063477,Y,22741795,CTCCTGTACTTGTTCATTAAATAATGATTCCTTGGATATACCAAGT...,0.795170
4,cg00121626,Y,21664296,AGGTGAATGAAGAGACTAATGGGAGTGGCTTGCAAGCCAGGTACTG...,0.398251
...,...,...,...,...,...
485507,ch.22.909671F,22,46114168,TTTTCCTTTTAGCTGCTGATAGATTAATAGTATGTGAACCTTTTAA...,0.056644
485508,ch.22.46830341F,22,48451677,TGTGCATACATGCGCATGTGAACAGTCCATGGAGCTTAATCCCCTG...,0.026932
485509,ch.22.1008279F,22,48731367,CTGGCAGGGCACACACCTCAGCTGGGCCCTGTGGCAGGTGAACCCC...,0.020596
485510,ch.22.47579720R,22,49193714,ATGTACCCATACGGGAAAGGCCGCGTGAAGATGGAGACAGAGATGG...,0.064398


In [3]:
refseq_map = {
    "1": "NC_000001.10",
    "2": "NC_000002.11",
    "3": "NC_000003.11",
    "4": "NC_000004.11",
    "5": "NC_000005.9",
    "6": "NC_000006.11",
    "7": "NC_000007.13",
    "8": "NC_000008.10",
    "9": "NC_000009.11",
    "10": "NC_000010.10",
    "11": "NC_000011.9",
    "12": "NC_000012.11",
    "13": "NC_000013.10",
    "14": "NC_000014.8",
    "15": "NC_000015.9",
    "16": "NC_000016.9",
    "17": "NC_000017.10",
    "18": "NC_000018.9",
    "19": "NC_000019.9",
    "20": "NC_000020.10",
    "21": "NC_000021.8",
    "22": "NC_000022.10",
    "X": "NC_000023.10",
    "Y": "NC_000024.9"
}

annotated['REFSEQ_chr'] = annotated['CHR'].map(refseq_map)

In [4]:
annotated['CHR'] = annotated['CHR'].astype(str)

In [6]:
annotated.to_csv('../data/me_chip/annotated_beta_values.csv', index=False)

## Accessing Windows

In [5]:
annotated.dtypes

IlmnID               object
CHR                  object
MAPINFO               int64
Forward_Sequence     object
mean_beta           float64
REFSEQ_chr           object
dtype: object

In [2]:
annotated = pd.read_csv('../data/me_chip/annotated_beta_values.csv', dtype={'CHR': str})

In [ ]:

genome = pysam.FastaFile('/home/tobyc/data/borzoi-pytorch/data/ref_genomes/GRCh37/GCF_000001405.13/GCF_000001405.13_GRCh37_genomic.fna')

annot_subset = annotated.iloc[:100]
for idx, row in annot_subset.iterrows():
    chrom, start, end = extract_window(row['REFSEQ_chr'], row['MAPINFO'])
    print(process_sequence(genome, chrom, start, end).shape)

(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524288, 4)
(524

## Splitting

In [8]:
annotated['CHR'].value_counts(normalize=True)

CHR
1     0.096510
6     0.075407
2     0.071698
7     0.061825
11    0.059306
17    0.057422
19    0.052565
3     0.051820
12    0.050543
10    0.050232
5     0.050106
16    0.045249
8     0.043150
4     0.042149
15    0.031429
14    0.031056
13    0.025303
X     0.023134
20    0.021377
9     0.020311
22    0.017614
18    0.012197
21    0.008739
Y     0.000857
Name: proportion, dtype: float64

In [9]:
annotated_train = annotated[~annotated['CHR'].isin(['3', '12', '5', '10'])]
annotated_val = annotated[annotated['CHR'].isin(['3', '12'])]
annotated_test = annotated[annotated['CHR'].isin(['5', '10'])]

In [11]:
annotated_train.to_csv('../data/me_chip/train.csv', index=False)
annotated_val.to_csv('../data/me_chip/val.csv', index=False)
annotated_test.to_csv('../data/me_chip/test.csv', index=False)

In [17]:
annotated_train = annotated[~annotated['CHR'].isin(['19', '16', '22', '21', '6', '21', '17', '13', '20'])]
annotated_val = annotated[annotated['CHR'].isin(['19', '16', '22', '21', '6'])]
annotated_test = annotated[annotated['CHR'].isin(['21', '17', '13', '20'])]

annotated_train.to_csv('../data/me_chip/ernest_train.csv', index=False)
annotated_val.to_csv('../data/me_chip/ernest_val.csv', index=False)
annotated_test.to_csv('../data/me_chip/ernest_test.csv', index=False)

## Predicting Multiple Values Per Window

Workflow:
- Separate genome into overlapping 524,288-length bins (intervals).
- Check if there is an Me site in the central 196,608 bp, otherwise check if it is in the central 262,144.
- If there is no Me site, drop the bin. If sites are outside the central 196,608, readjust window to contain them.
- Create windowed array using Me values, averaging if multiple are in the same window. 
- Save h5 file with window and 32bp-windowed Me values.
- Create hf dataset with index, coordinates and path to paired file.
- Shift window by 262,144 and repeat the process.

Example below is for chromosome 1 (NC_000001.10)

In [5]:
chr1_length = genome.get_reference_length('NC_000001.10')
genome.get_reference_length('NC_000001.10')

249250621

In [3]:
annotated = pd.read_csv('../data/me_chip/annotated_beta_values.csv', dtype={'CHR': str})
annotated_chr1 = annotated[annotated['CHR'] == '1'].sort_values(by='MAPINFO').set_index('MAPINFO')

In [4]:
annotated_train = annotated[~annotated['CHR'].isin(['19', '16', '22', '21', '6', '21', '17', '13', '20'])]
annotated_val = annotated[annotated['CHR'].isin(['19', '16', '22', '21', '6'])]
annotated_test = annotated[annotated['CHR'].isin(['21', '17', '13', '20'])]

In [5]:
stride = 262_144
centre_window = 196_608
window_offset = (stride - centre_window / 2)


for i in range(0, stride * 10, stride): # chr1_length - stride
    measured = annotated_chr1.loc[i + window_offset: i+window_offset + centre_window].copy()
    if measured.empty:
        print(f"No data for window starting at {i}")
    else:
        sequence = genome.fetch('NC_000001.10', i, i + 2 * stride)
        out_channel = np.zeros(stride // 16) # stride is half the sequence length.
        measured['bin_loc'] = (measured.index - i) // 32 # Gets a 32 bp resolution.
        for bin_loc, group in measured.groupby('bin_loc'):
            out_channel[bin_loc] = group['mean_beta'].mean() # type: ignore

        # display(measured)
        nonzero = out_channel[np.where(out_channel != 0)]    
        print(len(nonzero), len(measured))
        assert len(nonzero) <= len(measured)

No data for window starting at 0
21 22
134 151
394 410
327 353
212 227
174 181
346 356
350 367
77 77


In [6]:
seq_len = 524_288 # 2 * stride
target_len = centre_window // 32 # to match Borzoi output resolution.

for dataset, name in zip([annotated_train, annotated_val, annotated_test], ['train', 'val', 'test']):
    with h5py.File(f"../data/me_chip/ernest_{name}.h5", "w") as f:
        f.create_dataset(
            "sequence",
            shape=(0, 4, seq_len),
            maxshape=(None, 4, seq_len),
            dtype=np.uint8,
            compression='lzf',
            chunks=(1, 4, seq_len),            
        )
        f.create_dataset(
            "me_track",
            shape=(0, target_len),
            maxshape=(None, target_len),
            dtype=np.float32,
            compression="lzf",
            chunks=(1, target_len)
        )
        f.create_dataset("chromosome", shape=(0,), maxshape=(None,), dtype=h5py.string_dtype(encoding='utf-8'))
        f.create_dataset("start_coord", shape=(0,), maxshape=(None,), dtype=np.int32)

    with h5py.File(f"../data/me_chip/ernest_{name}.h5", "a") as f:
        for chrom in dataset['REFSEQ_chr'].unique():
            print(f"Processing chromosome {chrom}")

            chr_data = (
                dataset[dataset['REFSEQ_chr'] == chrom]
                .sort_values(by='MAPINFO')
                .set_index('MAPINFO')
            )
            chr_length = genome.get_reference_length(chrom)

            seqs, targets, chroms, starts = [], [], [], []

            for i in range(0, chr_length - stride, stride):
                measured = chr_data.loc[i + window_offset : i + window_offset + centre_window].copy()
                if measured.empty:
                    continue

                seq = process_sequence(genome, chrom, i, i + 2 * stride)
                out_channel = bin_methylation(measured, start_coordinate=i, seq_len=seq_len, resolution=32)

                seqs.append(seq)
                targets.append(out_channel)
                chroms.append(chrom)
                starts.append(i)

            if not seqs:
                continue

            # Convert to arrays for bulk writing
            seqs = np.stack(seqs)
            targets = np.stack(targets)
            chroms = np.array(chroms, dtype=h5py.string_dtype(encoding="utf-8"))
            starts = np.array(starts, dtype=np.int32)

            n_new = seqs.shape[0]
            start_idx = f["sequence"].shape[0]

            for name, data in zip(['sequence', 'me_track', 'chromosome', 'start_coord'], [seqs, targets, chroms, starts]):
                f[name].resize(start_idx + n_new, axis=0)
                f[name][-n_new:] = data
            

Processing chromosome NC_000024.9
Processing chromosome NC_000023.10
Processing chromosome NC_000001.10
Processing chromosome NC_000002.11
Processing chromosome NC_000003.11
Processing chromosome NC_000004.11
Processing chromosome NC_000005.9
Processing chromosome NC_000007.13
Processing chromosome NC_000008.10
Processing chromosome NC_000009.11
Processing chromosome NC_000010.10
Processing chromosome NC_000011.9
Processing chromosome NC_000012.11
Processing chromosome NC_000014.8
Processing chromosome NC_000015.9
Processing chromosome NC_000018.9
Processing chromosome NC_000006.11
Processing chromosome NC_000016.9
Processing chromosome NC_000019.9
Processing chromosome NC_000021.8
Processing chromosome NC_000022.10
Processing chromosome NC_000013.10
Processing chromosome NC_000017.10
Processing chromosome NC_000020.10
Processing chromosome NC_000021.8


In [10]:
196608 // 32

6144